# LSE ST451: Bayesian Machine Learning
## Author: Kostas Kalogeropoulos

## Week 5: Variational Bayes

Topics covered 
 - Mean field approximation
 - Automatic Differentiation Variational Inference (ADVI) in PyStan
 
**NOTE:** The PyStan package is not included with Anaconda. Use `python3 -m pip install pystan` to install it in the terminal or Anaconda prompt 

In [1]:
import pandas as pd
import numpy as np

### Mean Field approximation 

We will first simulate $100$ independent observations from the model 

$$
y\sim N(\mu, \tau^{-1})
$$

with $\mu=5$ and $\tau^{-1}=0.5$.

Then we will treat $\mu$ and $\tau^{-1}$ as unknown and will use the mean field approximation algorithm presented in the lecture to estimated them. 

#### Simulate Data

In [2]:
#Set parameters and simulate data
n = 100
mu = 3
tau = 2
std = np.sqrt(1/tau)
y = mu + std*np.random.randn(n)

#Set prior hyperparameters
mu0 = 0
lam0 = 1 #unit information prior
a0 = 0.001
b0 = 0.001


# get sufficient stats
Sy = np.sum(y)
Sy2 = np.sum(y**2)
m02 = mu0**2

#### Run the algorithm

In [3]:
#initialise parameters
muf = 0
tauf = 1
af = 1
bf = 1

#algorithmic parameters
maxiter = 1000
tol = 0.0000001

#objects to store the values of the parameters to be optimised.
Thetas = np.ones((maxiter,4))
Thetas[0,] = np.array([muf,tauf,af,bf])

#main while loop
i = 0
diff = 1 
while (i<maxiter) and (diff>tol):
    i = i+1
    af = a0+(n+1)/2
    Emu = muf   #E(mu)
    Emu2 = (1/tauf)+muf**2   #E(mu^2)
    bf = b0+0.5*(Sy2-2*muf*Sy+n*Emu2)+0.5*lam0*(m02 - 2*muf*mu0 + Emu2)
    tauf = (lam0+n)*af/bf   #E(tau)
    muf = (lam0*mu0+Sy)/(lam0+n)
    Thetas[i,] = np.array([muf,tauf,af,bf])
    dThetas = (Thetas[i,]-Thetas[i-1,])**2
    diff = np.max(dThetas)

#summarise output
muf = Thetas[i,0]
tauf = Thetas[i,1]
af = Thetas[i,2]
bf = Thetas[i,3]
print('Converged at ',i,' iterations')
results = np.array([[muf,af/bf,tauf],[np.mean(y),1/np.var(y),n/np.var(y)]])
col = ['muf','tauf','tau muf']
ind = ['VB','MLE']
results = pd.DataFrame(results,columns = col,index=ind)
results


Converged at  6  iterations


,muf,tauf,tau muf
VB,2.907077,1.777330,179.510288
MLE,2.936148,2.095219,209.521926


### Activity 1

Let $y=(y_1, \dots, y_n)$ be independent Poisson($\lambda$) observations. Assume that $\lambda$ follows the Gamma($2,\beta$) distribution, where $\beta$ follows the Exponential($1$) distribution. The aim is to draw inference from the posterior $\pi(\theta|y)$, where $\theta=(\lambda, \beta)$. 

#### Simulate data

In [4]:
#Set parameters and simulate data
n = 100
np.random.seed(1)
beta_true = np.random.exponential(1,1)
lambda_true  = np.random.gamma(2,1/beta_true,1)
print('beta value: ',beta_true,' lambda value: ',lambda_true)
y = np.random.poisson(lambda_true,n)

# get sufficient stats
sy = np.sum(y)
print('ybar: ',sy/n)

beta value:  [0.53960584]  lambda value:  [1.53955169]
ybar:  1.41


The variational Bayes algorithm **approximates** $\pi(\theta|y)$ using the mean field approximation 

$$
q(\theta|y, \phi)=q(\lambda|y, \phi)q(\beta|y, \phi)
$$

It can be shown (see exam paper of 2019, question 2a) that such an algorithm may consist of the following steps

 1. Initialise at $q(\lambda)$ to be the Gamma($a_{\lambda},b_{\lambda}$) and $q(\beta)$ to be the Gamma($a_{\beta},b_{\beta}$) distribution, setting $$a_{\lambda}=2+\sum_iy_i,\;\; b_{\lambda}=b_{\lambda}^0,\;\;\;a_{\beta}=3,\;\text{ and }\;\;b_{\beta}=b_{\beta}^0. $$
 2. Iteratively update $b_{\lambda}$ and $b_{\beta}$ until the parameters or the ELBO converge. At iteration $i$ will have:
 
    a. Set $$b_{\lambda}^{i}=n+\mathbb{E}_{q(\beta)}[\beta]=n+3/b_{\beta}^{i-1}$$
    
    b. Set $$b_{\beta}^{i}=1+\mathbb{E}_{q(\lambda)}[\lambda]=1+(2+\sum_iy_i)/b_{\lambda}^{i}$$


**Task:** Code the above algorithm and fit it to the simulated data. Check your answers in terms of the lambda estimates.

Put your code below

In [5]:
#initialise parameters
a_lam = 2+np.sum(y)
b_lam = 1 # this was set to an arbitrary value. feel free to try another one
a_bet = 3
b_bet = 1 # this was set to an arbitrary value. feel free to try another one

#algorithmic parameters
maxiter = 1000
tol = 0.0000001

#objects to store the values of the parameters to be optimised.
Thetas = np.ones((maxiter,4))
Thetas[0,] = np.array([a_lam,b_lam,a_bet,b_bet])

#main while loop
i = 0
diff = 1 
while (i<maxiter) and (diff>tol):
    i = i+1
    b_lam = n + (3/b_bet)
    b_bet = 1 + ((2+sy)/b_lam)
    Thetas[i,] = np.array([a_lam,b_lam,a_bet,b_bet])
    dThetas = (Thetas[i,]-Thetas[i-1,])**2
    diff = np.max(dThetas)

#summarise output
a_lam = Thetas[i,0]
b_lam = Thetas[i,1]
a_bet = Thetas[i,2]
b_bet = Thetas[i,3]
print('Converged at ',i,' iterations')

print('lambda VB mean: ',a_lam/b_lam, 'lambda MLE: ',sy/n,' true lambda ',lambda_true)

print('beta VB mean: ',a_bet/b_bet, ' true beta ',beta_true)

# Check VB for beta
beta_samples = np.random.gamma(a_bet,1/b_bet,100000)
np.percentile(beta_samples,99.5)
print('beta VB 95\% credible interval: ',np.percentile(beta_samples,2.5),np.percentile(beta_samples,97.5) )

Converged at  4  iterations
lambda VB mean:  1.412435556250598 lambda MLE:  1.41  true lambda  [1.53955169]
beta VB mean:  1.2435565344852542  true beta  [0.53960584]
beta VB 95\% credible interval:  0.25722643821840335 2.9981648745617258


## ADVI with PyStan

To load PyStan use the line below

In [6]:
import pystan as stan

To run ADVI for a model in PyStan the following steps are needed 

 1. Put the data in a dictionary
 2. Define the model and priors in a piece of text, using the intuitive [Stan language](https://mc-stan.org)
 3. Compile/build the model
 4. Execute the ADVI algorithm

We illustrate the above in the simple example we did in the beginning. First we simulate the data as before and add them in the Python dictionary `simple_model_data`

In [7]:
n = 100
mu_true = 3
tau_true = 2
std_true = np.sqrt(1/tau_true)
y = mu_true + std_true*np.random.randn(n)
simple_model_data = {"N": n,"y": y}

Next, we specify the model using the Stan language

In [8]:
simple_model_code = """
// The input data is a vector 'y' of length 'N'.
data {
  int<lower=0> N;
  real y[N];
}

// The parameters accepted by the model. Our model
// accepts two parameters 'mu' and 'sigma'.
parameters {
  real mu;
  real<lower=0> sigma;
}

// The model to be estimated. We model the output
// 'y' to be normally distributed with mean 'mu'
// and standard deviation 'sigma'.
model {
  mu ~ normal(0,1000);
  sigma ~ normal(0,100);
  for (i in 1:N)
    y[i] ~ normal(mu,sigma);
  //target += normal_lpdf(y| mu, sigma);
}
"""


In order to turn the text above into an stan model object we can use the code below (this may take a few seconds even up to a minute): 

In [9]:
fit = stan.StanModel(model_code=simple_model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_58b3f0de176abbb89d63843ba045506c NOW.


Now we can apply ADVI to the model above based on some data that we should also provide. We will do so with our simulated data `simple_model_data`.

In [10]:
fit_vb = fit.vb(data=simple_model_data)

The results are now in the `fit_vb` object. But this is not very user friendly, e.g. see below:

In [11]:
print(fit_vb)

OrderedDict([('args', {'random_seed': '1113118203', 'chain_id': 1, 'init': b'random', 'init_radius': 2.0, 'enable_random_init': False, 'append_samples': False, 'sample_file': b'/tmp/tmp9kz9gh5x/output.csv', 'method': 'VARIATIONAL', 'iter': 10000, 'grad_samples': 1, 'elbo_samples': 100, 'eval_elbo': 100, 'output_samples': 1000, 'eta': 1.0, 'adapt_engaged': True, 'adapt_iter': 50, 'tol_rel_obj': 0.01, 'algorithm': 'MEANFIELD'}), ('inits', [0.29587428525331827, 2.325641212178224]), ('sampler_params', [[2.961747282015544, 2.745242393833805, 2.7768233154523476, 2.9032944299060772, 2.8209861158328096, 2.8649123973388018, 2.9051433467286074, 3.014855363881597, 3.0587537076979636, 2.8126762279515005, 2.7995469921497143, 2.859701605449376, 2.925049386532395, 2.9518009586361713, 2.9818693795349556, 3.031335961422527, 2.836685135740418, 2.9517461499048734, 2.8503538475675065, 2.859334761191508, 2.9139614566344045, 2.861266068663441, 2.8805413748562048, 2.9277346371687014, 2.8450806801712587, 2.81

In [12]:
fit_vb['sampler_param_names']

['mu', 'sigma', 'lp__']

The following code and python function transforms the `fit_vb` results object into something slightly more friendly, i.e. and ordered Python dictionary

In [70]:
from collections import OrderedDict
def pystan_vb_extract(results):
    param_specs = results['sampler_param_names']
    samples = results['sampler_params']
    n = len(samples[0])

    # first pass, calculate the shape
    param_shapes = OrderedDict()
    for param_spec in param_specs:
        splt = param_spec.split('[')
        name = splt[0]
        if len(splt) > 1:
            idxs = [int(i) for i in splt[1][:-1].split(',')]  # no +1 for shape calculation because pystan already returns 1-based indexes for vb!
        else:
            idxs = ()
        param_shapes[name] = np.maximum(idxs, param_shapes.get(name, idxs))

    # create arrays
    params = OrderedDict([(name, np.nan * np.empty((n, ) + tuple(shape))) for name, shape in param_shapes.items()])

    # second pass, set arrays
    for param_spec, param_samples in zip(param_specs, samples):
        splt = param_spec.split('[')
        name = splt[0]
        if len(splt) > 1:
            idxs = [int(i) - 1 for i in splt[1][:-1].split(',')]  # -1 because pystan returns 1-based indexes for vb!
        else:
            idxs = ()
        params[name][(..., ) + tuple(idxs)] = param_samples

    return params

In [71]:
results=pystan_vb_extract(fit_vb)
results

OrderedDict([('mu',
              array([2.98224272, 2.89286635, 2.89868312, 2.9483607 , 3.0304715 ,
                     2.90391955, 2.83607857, 2.97730785, 2.94718927, 2.9190666 ,
                     2.85411763, 2.95975983, 3.01656103, 2.80762043, 2.79881159,
                     2.80174971, 2.8978524 , 2.98932757, 2.91933928, 2.90996836,
                     2.87490641, 2.91466959, 2.93901804, 2.95847548, 2.9152755 ,
                     2.88897345, 2.89695438, 3.00005303, 2.83938503, 2.86421369,
                     2.90610659, 2.85131083, 2.95136381, 3.01827345, 2.88121965,
                     2.85252414, 2.93621562, 2.76317614, 2.8320867 , 2.91755736,
                     3.05697896, 2.9510203 , 2.92302331, 2.96825197, 2.90179475,
                     2.83664081, 3.00591099, 2.88288834, 3.11566396, 3.01471941,
                     2.87096305, 2.85168548, 2.80571539, 2.95684598, 2.95055769,
                     2.95423327, 2.85346921, 2.9722857 , 3.06609144, 2.97638746,
        

Now we can access specific elements, e.g. samples from the variational approximate posterior of `mu` and `sigma` based on the ADVI mean field approximation. For example, samples from the marginal approximate posterior of `mu` can be obtained with the code below:

In [72]:
print("results['mu']: ", results['mu'])

results['mu']:  [2.98224272 2.89286635 2.89868312 2.9483607  3.0304715  2.90391955
 2.83607857 2.97730785 2.94718927 2.9190666  2.85411763 2.95975983
 3.01656103 2.80762043 2.79881159 2.80174971 2.8978524  2.98932757
 2.91933928 2.90996836 2.87490641 2.91466959 2.93901804 2.95847548
 2.9152755  2.88897345 2.89695438 3.00005303 2.83938503 2.86421369
 2.90610659 2.85131083 2.95136381 3.01827345 2.88121965 2.85252414
 2.93621562 2.76317614 2.8320867  2.91755736 3.05697896 2.9510203
 2.92302331 2.96825197 2.90179475 2.83664081 3.00591099 2.88288834
 3.11566396 3.01471941 2.87096305 2.85168548 2.80571539 2.95684598
 2.95055769 2.95423327 2.85346921 2.9722857  3.06609144 2.97638746
 2.82606881 2.88852345 2.9702255  2.98481554 2.91112848 2.84916016
 2.92873005 2.85497191 2.94157827 2.84648471 2.87853644 2.91491893
 2.9929227  2.96678386 2.94593811 3.01037891 2.91810344 2.96323392
 2.85893513 2.97900925 2.8299741  2.85322138 2.9475859  2.97994944
 2.92866532 2.94578068 2.86486874 2.886283   2.

We can now check the ADVI approximation against the values we simulated data from

In [73]:
mu=results['mu']
print('mu_true: ',mu_true,)
print('vb posterior mean of mu: ',np.mean(mu))
sigma=results['sigma']
print('sigms_true: ',std,)
print('vb posterior mean of sigma: ',np.mean(sigma))

mu_true:  3
vb posterior mean of mu:  2.9312316274773402
sigms_true:  0.7071067811865476
vb posterior mean of sigma:  0.6649482290378088


### Activity 2

For the activity we will work with the dataset of last week that contains cases from a study that was conducted between 1958 and 1970 at the University of Chicago's Billings Hospital on the survival of patients who had undergone surgery for breast cancer.

Four columns 

1. Age of patient at time of operation (numerical)
2. Patient's year of operation (year - 1900, numerical)
3. Number of positive axillary nodes detected (numerical)
4. Survival status (class attribute)
     1 = the patient survived 5 years or longer
     2 = the patient died within 5 year

See more info at the UCI repository [page](https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.names). 

In [74]:
url_hospital = "https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data"
data1 = pd.read_csv(url_hospital, header=None,
        names=['age', 'year', 'nodes_detected', 'survival_status'])
data1.head()

,age,year,nodes_detected,survival_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1


For this activity we will fit a model with only the `nodes_detected` as input. Normal priors can be assigned on the $\beta$ coefficients with zero mean and some reasonably large variance -say $100$.

The task is to obtain the mean field Variational Bayes approximation to the posterior of $\beta$ using ADVI in RStan.  
The model can be written as

$$
y_i\sim Bernoulli(\pi_i),\;\; i=1,...,N,
$$
$$
\pi_i = \text{logit}^{-1}(\beta_0+\beta_1 x_{i}),
$$
$$
\beta_0 \sim N(0,100), \;  \beta_1 \sim N(0,100),
$$

where $y_i$ denotes the survival status and $x_{i}$ the nodes detected for the person $i$.


#### Hints for the activity

The data can be set up as below, by first creating the dictionary

In [76]:
data1['y'] = data1.survival_status.replace(2, 0).values
data = {"N": len(data1['y']),"y": data1['y'], "x": data1['nodes_detected']}

The code for defining the model in the Stan language consists of at least three parts
 
 1. the data part
 2. the parameters part
 3. the model part
 
For the data, the relevant code bit in Stan language can then be something like

`
data {
  int<lower=0> N;
  int<lower=0,upper=1> y[N];
  vector[N] x;
}
`

For the parameters, the requested model has two parameters, i.e. the `beta` coefficients, so the relevant code bit in the Stan language can be something like

`
parameters {
  vector[2] beta;
}
`

Finally, for the model part, you can specify the priors as before. For the `y[i]`'s note that these are now binary and a the function `bernoulli_logit()` is a combination of the logistic regression formulations. For example `y ~ bernoulli_logit(A)` for $A\in \mathbb{R}$, corresponds to

$$
y \sim Bernoulli(\pi),
$$
$$
\pi = \text{logit}^{-1}(A).
$$


**Code for the activity**

In [49]:
LR_model_code = """
// The input data is a vector 'y' of length 'N' consisting of binary variables.
// and the a vector x for the covariate 'nodes_detected'
data {
  int<lower=0> N;
  int<lower=0,upper=1> y[N];
  vector[N] x;
}

// The parameters accepted by the model. Our model
// contains the beta coefficients as parameters.
parameters {
  vector[2] beta;
}

// The model to be estimated. 
model {
  beta ~ normal(0,100);
  //target += bernoulli_logit_lpmf(y | beta[1] + beta[2] * x);
  for (n in 1:N)
    //y[n] ~ bernoulli(inv_logit(beta[1] + beta[2] * x[n]));
    y[n] ~ bernoulli_logit(beta[1]+ beta[2] * x[n]);
}
"""


In [50]:
LR_model = stan.StanModel(model_code=LR_model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f085df2471e28698a46941551b6dbf1e NOW.


In [55]:
LR_model_vb = LR_model.vb(data=data)

In [56]:
results=pystan_vb_extract(LR_model_vb)

In [57]:
beta=results['beta']
print('beta ADVI posterior means: ',np.mean(beta,axis=0))
#print('vb posterior mean of mu: ',np.mean(mu))

beta ADVI posterior means:  [ 1.40666436 -0.0909466 ]
